In [11]:
import dash
from dash import dcc
from dash import html
from datetime import datetime as dt
from datetime import date, timedelta
import yfinance as yf
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

app = dash.Dash(__name__)
server = app.server

item1 = html.Div(
    [
        html.H1(children="Stock Data Indicator", className="start", style={'textAlign': 'center', 'color': '#FF0000'}),
        html.Div([
            dcc.Input(id='stockcode', value='aapl', type='text', placeholder='Input stock code'),
            html.Button(id='stockcode_submit', type='submit', children='Submit', n_clicks=0)
        ], className='stockcodeselect', style={'textAlign': 'center'}),
        html.Div([
            html.Div([
                dcc.DatePickerRange(
                    id='my-date-picker-range',
                    initial_visible_month=date.today(),
                    end_date=date.today()
                ),
                html.Div(id='output-container-date-picker-range')
            ], className='datepicker', style={'textAlign': 'center'}),
            html.Div([html.Button(id='stock_price_click', type='button', children='STOCK PRICE', n_clicks=0, className='dateButton')], className='datepickerbutton')
        ], className='Date', style={'textAlign': 'center'}),
    ],
    className="nav"
)

info = html.Div(
    [
        html.Div(
            [html.Img(id='company_logo'),  # Logo
             html.H2(id='company_name')  # Company Name
             ],
            className="header"),
        html.Div(
            id="description", className="description_ticker")
    ],
    className="content", style={'textAlign': 'center'}
)

plots = html.Div([
    dcc.Graph(
        id="graphs-content"),
], className="plots")

item2 = html.Div(children=[info, plots], className='info')

app.layout = html.Div(children=[item1, item2], className='container')

def get_more(df):
    df['EWA_20'] = df['Close'].ewm(alpha=0.6).mean()
    fig = px.scatter(df,
                     x='Date',  # Date str,
                     y='EWA_20',  # EWA_20 str,
                     title="Exponential Moving Average vs Date")
    fig.update_traces(mode='lines')
    return fig

def get_stock_price_fig(df):
    df['EWA'] = df['Close'].ewm(alpha=0.5).mean()
    fig = px.line(df,
                  x='Date',  # Date str,
                  y=['Open', 'Close', 'EWA'],  # list of 'Open' and 'Close',
                  title="Closing, Opening and Indicator Price vs Date")
    return fig

@app.callback(
    Output(component_id='company_name', component_property='children'),
    Output('company_logo', 'src'),
    Output(component_id='description', component_property='children'),
    Input('stockcode_submit', 'n_clicks'),
    State(component_id='stockcode', component_property='value')
)
def update_company_details(n, val):
    ticker = yf.Ticker(val)
    inf = ticker.info
    df = pd.DataFrame().from_dict(inf, orient="index").T
    return df['shortName'][0], df['logo_url'][0], df['longBusinessSummary'][0]

@app.callback(
    Output('graphs-content', 'figure'),
    Input('stock_price_click', 'n_clicks'),
    Input('stockcode_submit', 'n_clicks'),
    State(component_id='stockcode', component_property='value'),
    State('my-date-picker-range', 'start_date'),
    State('my-date-picker-range', 'end_date')
)
def update_plots(n, m, val, start_date, end_date):
    ticker = yf.Ticker(val)
    df = ticker.history(start=start_date, end=end_date)
    df.reset_index(inplace=True)
    fig = get_stock_price_fig(df)
    return fig

if __name__ == '__main__':
    app.run_server(port=4050)


In [2]:
help(dash.html)

Help on package dash.html in dash:

NAME
    dash.html - Vanilla HTML components for Dash

PACKAGE CONTENTS
    A
    Abbr
    Acronym
    Address
    Area
    Article
    Aside
    Audio
    B
    Base
    Basefont
    Bdi
    Bdo
    Big
    Blink
    Blockquote
    Br
    Button
    Canvas
    Caption
    Center
    Cite
    Code
    Col
    Colgroup
    Content
    Data
    Datalist
    Dd
    Del
    Details
    Dfn
    Dialog
    Div
    Dl
    Dt
    Em
    Embed
    Fieldset
    Figcaption
    Figure
    Font
    Footer
    Form
    Frame
    Frameset
    H1
    H2
    H3
    H4
    H5
    H6
    Header
    Hgroup
    Hr
    I
    Iframe
    Img
    Ins
    Kbd
    Keygen
    Label
    Legend
    Li
    Link
    Main
    MapEl
    Mark
    Marquee
    Meta
    Meter
    Nav
    Nobr
    Noscript
    ObjectEl
    Ol
    Optgroup
    Option
    Output
    P
    Param
    Picture
    Plaintext
    Pre
    Progress
    Q
    Rb
    Rp
    Rt
    Rtc
    Ruby
    S
    Samp
    Scri

In [3]:
!pip install yfinance

  Using cached yfinance-0.2.41-py2.py3-none-any.whl.metadata (11 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
  Using cached peewee-3.17.6.tar.gz (3.0 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/73.5 kB ? eta -:--:--
   --------------------------------- ------ 61.4/73.5 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 73.5/73.5 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/112.2 kB ? eta -:--:--
   ---------------------------------------- 112.2/112.2 kB 3.3 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.6-py3-none-any.whl size=138938 sha256=dcfad08164190a5303de28

In [5]:
!pip install dash-bootstrap-components

   ---------------------------------------- 0.0/222.5 kB ? eta -:--:--
   - -------------------------------------- 10.2/222.5 kB ? eta -:--:--
   - -------------------------------------- 10.2/222.5 kB ? eta -:--:--
   ---------- ---------------------------- 61.4/222.5 kB 544.7 kB/s eta 0:00:01
   ---------------------------------- ----- 194.6/222.5 kB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 222.5/222.5 kB 1.2 MB/s eta 0:00:00
